In [5]:
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import re
import feather
from unidecode import unidecode

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%matplotlib inline

# Unzip files

In [12]:
all_zips = Path('/home/jeronimo/Escritorio/ctgen/notebooks/input_data/zips_contratos_INAI/')

for z in all_zips.iterdir():
    zip_ref = zipfile.ZipFile(z.as_posix(), 'r')
    zip_ref.extractall(all_zips.as_posix().replace('zips_', ''))
    zip_ref.close()

# Parsear contratos

## Prueba

In [46]:
contrato = pd.read_csv('/home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_INAI/contrato_Comision Federal de Competencia Economica_1.csv', 
                       encoding='latin1')
contrato.reset_index(inplace=True)
print(contrato.shape)
contrato.head()

(854, 13)


,level_0,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10,level_11,CONTRATACIONES
0,Institución,Número de Contrato,Procedimiento de contratación,Nombre de la persona a que se asignó el contrato,Fecha de celebración del Contrato,Monto del Contrato,Unidad Administrativa que celebró el contrato,Objeto del Contrato,Fecha Inicio del Contrato,Fecha de terminación del contrato,Documento: Sitio en Internet,Convenio Modificatorio,NaN
1,"="" Comisión Federal de Competencia. Ahora Comi...","="" 10111001-003-11-55""","="" LICITACION PUBLICA NACIONAL""","="" EFINFO S.A.P.I. DE C.V.""","="" 24/02/2011""","="" 440800.0""","="" DIRECCIÓN GENERAL DE ADMINISTRACIÓN""","="" SERVICIO DE MONITOREO ELECTRONICO Y SUMINIS...","="" 01/03/2011""","="" 31/12/2011""","="" http://""","="" SI""",NaN
2,"="" Comisión Federal de Competencia. Ahora Comi...","="" CFC-001-2011""","="" LICITACION PUBLICA NACIONAL""","="" SANDRA SEGURA QUIROZ""","="" 23/03/2011""","="" 226085.89""","="" DIRECCIÓN GENERAL DE ADMINISTRACIÓN""","="" PAPELERIA Y SUMINISTROS DE OFICINA min 94 ...","="" 23/03/2011""","="" 31/12/2011""","="" http://""","="" """,NaN
3,"="" Comisión Federal de Competencia. Ahora Comi...","="" CFC-002-2011""","="" LICITACION PUBLICA NACIONAL""","="" AUTO GLASS S.A. DE C.V.""","="" 23/03/2011""","="" 21269.76""","="" DIRECCIÓN GENERAL DE ADMINISTRACIÓN""","="" Adquisición de Papeleria y Suministro de of...","="" 23/03/2011""","="" 31/12/2011""","="" http://""","="" """,NaN
4,"="" Comisión Federal de Competencia. Ahora Comi...","="" CFC-003-2011""","="" LICITACION PUBLICA NACIONAL""","="" OFFICE CLUB S.A. DE C.V.""","="" 23/03/2011""","="" 3403.21""","="" DIRECCIÓN GENERAL DE ADMINISTRACIÓN""","="" Adquisición de Papeleria y Suministro de of...","="" 23/03/2011""","="" 31/12/2011""","="" http://""","="" """,NaN


In [47]:
# Los nombres de columnas están en el primer renglón, excepto CONTRATACIONES que se queda como nombre de columna
contrato.columns = contrato[0:1].values[0].tolist()[0:-1] + [contrato.columns[-1]]

# Tiramos el renglón con os nombres
contrato.drop(0, inplace=True)

In [129]:
def parse_inai(string):
    if type(string)==str:
        try:
            s = string.replace('=', '').replace('\"', '').strip()
            return s
        except ValueError or AttributeError:
            pass
    else: 
        return string

In [130]:
parsed_contrato = contrato.applymap(parse_inai)
print(parsed_contrato.shape)
parsed_contrato.head()

(45939, 13)


,Institución,Número de Contrato,Procedimiento de contratación,Nombre de la persona a que se asignó el contrato,Fecha de celebración del Contrato,Monto del Contrato,Unidad Administrativa que celebró el contrato,Objeto del Contrato,Fecha Inicio del Contrato,Fecha de terminación del contrato,Documento: Sitio en Internet,Convenio Modificatorio,CONTRATACIONES
1,CONSEJO DE PROMOCIÓN TURÍSTICA DE MÉXICO S.A....,C005/2013,ADJUDICACION DIRECTA,SESITI S.C.,01/03/2013,161036.0,DIRECCIÓN DE MERCADEO PERSONALIZADO E INTERNET,Servicio técnico especializado para el servici...,01/03/2013,31/12/2013,http://,,NaN
2,CONSEJO DE PROMOCIÓN TURÍSTICA DE MÉXICO S.A....,C008/2013,ADJUDICACION DIRECTA,PEDRO ROBLES OTERO DESPACHO LABORAL,06/03/2013,317982.0,DIRECCIÓN EJECUTIVA DE ASUNTOS JURÍDICOS,Contratación de los servicios profesionales pa...,06/03/2013,31/12/2013,http://,,NaN
3,CONSEJO DE PROMOCIÓN TURÍSTICA DE MÉXICO S.A....,C001/2013,ADJUDICACION DIRECTA,TAYIRA TRAVEL S.A. DE C.V.,14/01/2013,9000000.0,DIRECCIÓN DE RECURSOS MATERIALES,Reservación expedición de pasajes aéreos naci...,14/01/2013,31/12/2013,http://,,NaN
4,CONSEJO DE PROMOCIÓN TURÍSTICA DE MÉXICO S.A....,P004/2013,ADJUDICACION DIRECTA,FERMIN FERNÁNDEZ UNANUE,31/01/2013,120000.0,DIRECCIÓN DE RECURSOS MATERIALES,Impresión y/o grabado de papelería tarjetas d...,01/02/2013,31/12/2013,http://,,NaN
5,CONSEJO DE PROMOCIÓN TURÍSTICA DE MÉXICO S.A....,P002/2013,ADJUDICACION DIRECTA,GRUPO VAN DER CORP S.A. DE C.V.,24/01/2013,378000.0,DIRECCIÓN DE RECURSOS MATERIALES,Servicio integral de audio y video con person...,24/01/2013,31/12/2013,http://,,NaN


## Todo

In [132]:
def parse_inai(string):
    if type(string)==str:
        try:
            s = string.replace('=', '').replace('\"', '').strip()
            return s
        except ValueError or AttributeError:
            pass
    else: 
        return string

In [133]:
all_files = Path('/home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_no_parseados_INAI/')
expr = re.compile('\_.*\_')

In [134]:
ordered_dirs = [f.as_posix() for f in all_files.iterdir()]
ordered_dirs = sorted(ordered_dirs)
ordered_dirs[0:5]

['/home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_no_parseados_INAI/contrato_Agricultura, Ganaderia, Desarrollo Rural, Pesca y Alimentacion_1.csv',
 '/home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_no_parseados_INAI/contrato_Comision Federal de Competencia Economica_1.csv',
 '/home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_no_parseados_INAI/contrato_Comunicaciones y Transportes_1.csv',
 '/home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_no_parseados_INAI/contrato_Comunicaciones y Transportes_2.csv',
 '/home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_no_parseados_INAI/contrato_Consejeria Juridica del Ejecutivo Federal_1.csv']

In [152]:
errorfiles = []

for f in ordered_dirs:
    
    s = expr.search(f.split('contratos_no_parseados_INAI/')[-1]).group(0)[1:-1]
    
    #Salvamos en la carpeta nueva
    dirname = '/home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_INAI/'
    ss = s.replace(' ', '_').replace(',', '').lower()
    n_file = dirname+'contrato_%s.csv' %ss
    
    try:
        contrato = pd.read_csv(f, encoding='latin1').reset_index()

        # Los nombres de columnas están en el primer renglón, excepto CONTRATACIONES que se queda como nombre de columna
        contrato.columns = contrato[0:1].values[0].tolist()[0:-1] + [contrato.columns[-1]]

        # Tiramos el renglón con los nombres
        contrato.drop(0, inplace=True)

        parsed_contrato = contrato.applymap(parse_inai)
        
        if Path(n_file).is_file():
            # Do not overwrite if exists
            parsed_contrato.to_csv(n_file, mode='a', index=False, header=False)
            print('Added to %s' %(n_file))
        else:
            # Si ya existía no queremos el header
            parsed_contrato.to_csv(n_file, mode='a', index=False)
            print('Saved to %s' %(n_file))

    except ValueError:
        errorfiles.append(f)
        print('ERROR in : ', f)

Saved to /home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_INAI/contrato_agricultura_ganaderia_desarrollo_rural_pesca_y_alimentacion.csv
Saved to /home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_INAI/contrato_comision_federal_de_competencia_economica.csv
Saved to /home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_INAI/contrato_comunicaciones_y_transportes.csv
Added to /home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_INAI/contrato_comunicaciones_y_transportes.csv
Saved to /home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_INAI/contrato_consejeria_juridica_del_ejecutivo_federal.csv
Saved to /home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_INAI/contrato_consejo_nacional_de_ciencia_y_tecnologia.csv
Added to /home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_INAI/contrato_consejo_nacional_de_ciencia_y_tecnologia.csv
Saved to /home/jeronimo/Escritorio/ctgen/notebooks/input_data/contratos_INAI/contrato_

In [137]:
# No hubo errores al parsear
errorfiles

[]

# Sanity checks

In [158]:
aux = [f.as_posix() for f in all_files.iterdir() if f.as_posix().find('Salud_')!=-1]
imss = []

for f in aux:
    contrato = pd.read_csv(f, encoding='latin1').reset_index()

    # Los nombres de columnas están en el primer renglón, excepto CONTRATACIONES que se queda como nombre de columna
    contrato.columns = contrato[0:1].values[0].tolist()[0:-1] + [contrato.columns[-1]]

    # Tiramos el renglón con los nombres
    contrato.drop(0, inplace=True)

    parsed_contrato = contrato.applymap(parse_inai)
    imss.append(parsed_contrato)

imss = pd.concat(imss, ignore_index=True)
print(imss.shape)
imss.head()

In [159]:
imss.shape

(181383, 13)

In [160]:
imss.head()

,Institución,Número de Contrato,Procedimiento de contratación,Nombre de la persona a que se asignó el contrato,Fecha de celebración del Contrato,Monto del Contrato,Unidad Administrativa que celebró el contrato,Objeto del Contrato,Fecha Inicio del Contrato,Fecha de terminación del contrato,Documento: Sitio en Internet,Convenio Modificatorio,CONTRATACIONES
0,INSTITUTO NACIONAL DE CARDIOLOGÍA IGNACIO CHÁVEZ,C03577-24,ADJUDICACION DIRECTA,PRODUCTOS QUIMICOS MM S.A. DE C.V.,23/03/2010,742.55,SUBDIRECCIÓN DE RECURSOS MATERIALES,Introductor transradial 6Fr. Marca Cordis Ca...,23/03/2010,23/03/2010,,,NaN
1,INSTITUTO NACIONAL DE CARDIOLOGÍA IGNACIO CHÁVEZ,C03577-25,ADJUDICACION DIRECTA,PRODUCTOS QUIMICOS MM S.A. DE C.V.,23/03/2010,445.53,SUBDIRECCIÓN DE RECURSOS MATERIALES,Cateter Dx 6FR JL 3.5 Mca. Cordis Cat. 534-618T,23/03/2010,23/03/2010,,,NaN
2,INSTITUTO NACIONAL DE CARDIOLOGÍA IGNACIO CHÁVEZ,C03577-26,ADJUDICACION DIRECTA,PRODUCTOS QUIMICOS MM S.A. DE C.V.,23/03/2010,1633.62,SUBDIRECCIÓN DE RECURSOS MATERIALES,Catéter guía JL3.5 6Fr. Marca Cordis Catálog...,23/03/2010,23/03/2010,,,NaN
3,INSTITUTO NACIONAL DE CARDIOLOGÍA IGNACIO CHÁVEZ,C03577-27,ADJUDICACION DIRECTA,PRODUCTOS QUIMICOS MM S.A. DE C.V.,23/03/2010,33576.21,SUBDIRECCIÓN DE RECURSOS MATERIALES,Stent Cypher 3.0x28mm Marca Cordis Catálogo ...,23/03/2010,23/03/2010,,,NaN
4,INSTITUTO NACIONAL DE CARDIOLOGÍA IGNACIO CHÁVEZ,C03577-28,ADJUDICACION DIRECTA,PRODUCTOS QUIMICOS MM S.A. DE C.V.,23/03/2010,9039.66,SUBDIRECCIÓN DE RECURSOS MATERIALES,Catéter balón 3.0/15 mm Mca. Cordis Mod. Dur...,23/03/2010,23/03/2010,,,NaN


In [161]:
imss2 = pd.read_csv('input_data/contratos_INAI/contrato_salud.csv')
print(imss2.shape)
imss2.head()

(181383, 13)


,Institución,Número de Contrato,Procedimiento de contratación,Nombre de la persona a que se asignó el contrato,Fecha de celebración del Contrato,Monto del Contrato,Unidad Administrativa que celebró el contrato,Objeto del Contrato,Fecha Inicio del Contrato,Fecha de terminación del contrato,Documento: Sitio en Internet,Convenio Modificatorio,CONTRATACIONES
0,Administración del Patrimonio de la Beneficenc...,APBP-003/2012,ADJUDICACION DIRECTA,JOHNATTAN ANTONIO CARDOSO GONZALEZ,25/01/2012,34526.0,APBP ADMINISTRACION DEL PATRIMONIO DE LA BENEF...,MANTENIMIENTO PREVENTIVO Y CORRECTIVO DEL PARQ...,27/01/2012,31/12/2012,NaN,SI,NaN
1,Administración del Patrimonio de la Beneficenc...,F109-07,ADJUDICACION DIRECTA,HUERTA MÉDICA MEXICANA S.A. DE C.V.,21/06/2007,25254.0,APBP ADMINISTRACION DEL PATRIMONIO DE LA BENEF...,PRÓTESIS DE RODILLA,21/06/2007,25/06/2007,NaN,NaN,NaN
2,Administración del Patrimonio de la Beneficenc...,F108-07,ADJUDICACION DIRECTA,SMITH & NEPHEW S.A. DE C.V.,21/06/2007,22035.0,APBP ADMINISTRACION DEL PATRIMONIO DE LA BENEF...,PRÓTESIS DE RODILLA,21/06/2007,03/07/2007,NaN,NaN,NaN
3,Administración del Patrimonio de la Beneficenc...,F107-07,ADJUDICACION DIRECTA,STRYKER MÉXICO S.A. DE C.V.,21/06/2007,26450.0,APBP ADMINISTRACION DEL PATRIMONIO DE LA BENEF...,PRÓTESIS DE CADERA,21/06/2007,03/07/2007,NaN,NaN,NaN
4,Administración del Patrimonio de la Beneficenc...,F106-07,ADJUDICACION DIRECTA,BIOMET MÉXICO S.A. DE C.V.,21/06/2007,32969.93,APBP ADMINISTRACION DEL PATRIMONIO DE LA BENEF...,PRÓTESIS DE CADERA,21/06/2007,26/06/2007,NaN,NaN,NaN


In [162]:
# No falta nada
imss2[~imss2.Institución.isin(imss.Institución)]

,Institución,Número de Contrato,Procedimiento de contratación,Nombre de la persona a que se asignó el contrato,Fecha de celebración del Contrato,Monto del Contrato,Unidad Administrativa que celebró el contrato,Objeto del Contrato,Fecha Inicio del Contrato,Fecha de terminación del contrato,Documento: Sitio en Internet,Convenio Modificatorio,CONTRATACIONES


In [163]:
# No perdimos contratos
imss2[~imss2['Número de Contrato'].isin(imss['Número de Contrato'])]

,Institución,Número de Contrato,Procedimiento de contratación,Nombre de la persona a que se asignó el contrato,Fecha de celebración del Contrato,Monto del Contrato,Unidad Administrativa que celebró el contrato,Objeto del Contrato,Fecha Inicio del Contrato,Fecha de terminación del contrato,Documento: Sitio en Internet,Convenio Modificatorio,CONTRATACIONES


In [164]:
# Subimos a s3
# !aws s3 cp --recursive input_data/contratos_INAI/ s3://opi-compranet/processed/contratos_INAI/

upload: input_data/contratos_INAI/contrato_comision_federal_de_competencia_economica.csv to s3://opi-compranet/processed/contratos_INAI/contrato_comision_federal_de_competencia_economica.csv
upload: input_data/contratos_INAI/contrato_consejeria_juridica_del_ejecutivo_federal.csv to s3://opi-compranet/processed/contratos_INAI/contrato_consejeria_juridica_del_ejecutivo_federal.csv
upload: input_data/contratos_INAI/contrato_comunicaciones_y_transportes.csv to s3://opi-compranet/processed/contratos_INAI/contrato_comunicaciones_y_transportes.csv
upload: input_data/contratos_INAI/contrato_defensa_nacional.csv to s3://opi-compranet/processed/contratos_INAI/contrato_defensa_nacional.csv
upload: input_data/contratos_INAI/contrato_agricultura_ganaderia_desarrollo_rural_pesca_y_alimentacion.csv to s3://opi-compranet/processed/contratos_INAI/contrato_agricultura_ganaderia_desarrollo_rural_pesca_y_alimentacion.csv
upload: input_data/contratos_INAI/contrato_consejo_nacional_de_ciencia_y_tecnologia.c

# Relación con tabla procedimientos

- CODIGO_DE_CONTRATO $\longleftrightarrow$ Número de contrato **????**

In [6]:
proceds = feather.read_dataframe('input_data/procedimientos_2017_06_14.feather')
proceds.head()

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO
0,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,RAUL EUGENIO ALVAREZ CARRASCO,HABILITADO,MICRO,512930.40,NaN,MXN,IA-901024986-N44-2012,None,INVITACION A CUANDO MENOS 3 PERSONAS,208993,N44 MOB ETC,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23,NaT,2012-10-23,2012-12-31,271313,ADQUISICION DE MOBILIARIO,Z15122015 04. Invitación a cuando menos tres N...,None,PoC,None,No,MX,NaN,None,NaN
1,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,MA DE LA LUZ TISCARENO ELIAS,HABILITADO,NO MIPYME,1887409.40,NaN,MXN,IA-901024986-N44-2012,None,INVITACION A CUANDO MENOS 3 PERSONAS,208992,N44 MOB ETC,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-10-17 14:51:00,2012-10-23 11:30:00,2012-10-23,NaT,2012-10-23,2012-10-31,271313,ADQUISICION DE MOBILIARIO,Z15122015 04. Invitación a cuando menos tres N...,None,PoC,None,No,MX,NaN,None,NaN
2,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,PAPELERIA CONSUMIBLES Y ACCESORIOS SA DE CV,HABILITADO,MICRO,1892535.82,NaN,MXN,IA-901024986-N53-2012,None,INVITACION A CUANDO MENOS 3 PERSONAS,257525,IEA.DJ.CONT. 1140/2012,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-11-05 14:34:00,2012-11-12 10:00:00,2012-10-19,NaT,2012-11-12,2012-12-31,268049,IEA PAPELERIA Y CONSUMIBLES,Z15122015 04. Invitación a cuando menos tres N...,None,PoC,None,No,MX,NaN,None,NaN
3,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761,TELECOMUNICACIONES MODERNAS SA DE CV,HABILITADO,PEQUENA,24234.00,NaN,MXN,AA-901024986-N31-2012,None,ADJUDICACION DIRECTA FEDERAL,241054,CONSUMIBLES,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-09-06 08:49:00,2012-09-07 09:00:00,NaT,NaT,2012-09-11,2012-12-31,252725,COORDINACION ESTATAL DE ASESORIA Y SEGUIMIENTO...,Z15122015 07. Adjudicación Directa Nacional Ar...,None,PoC,None,No,MX,NaN,None,NaN
4,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,19761,TELECOMUNICACIONES MODERNAS SA DE CV,HABILITADO,PEQUENA,18700.00,NaN,MXN,AA-901024986-N64-2012,None,ADJUDICACION DIRECTA FEDERAL,474294,ADQUISICION DE EQUIPO DE COMPUTO,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-12-12 11:38:00,2012-12-14 10:00:00,NaT,NaT,2012-12-17,2012-12-31,320854,EQUIPO DE COMPUTO DESARROLLO EDUCATIVO,Z15122015 07. Adjudicación Directa Nacional Ar...,None,PoC,None,No,MX,NaN,None,NaN


In [7]:
imss = pd.read_csv('input_data/contratos_INAI/contrato_salud.csv')
print(imss.shape)
imss.head()

(181383, 13)


,Institución,Número de Contrato,Procedimiento de contratación,Nombre de la persona a que se asignó el contrato,Fecha de celebración del Contrato,Monto del Contrato,Unidad Administrativa que celebró el contrato,Objeto del Contrato,Fecha Inicio del Contrato,Fecha de terminación del contrato,Documento: Sitio en Internet,Convenio Modificatorio,CONTRATACIONES
0,Administración del Patrimonio de la Beneficenc...,APBP-003/2012,ADJUDICACION DIRECTA,JOHNATTAN ANTONIO CARDOSO GONZALEZ,25/01/2012,34526.0,APBP ADMINISTRACION DEL PATRIMONIO DE LA BENEF...,MANTENIMIENTO PREVENTIVO Y CORRECTIVO DEL PARQ...,27/01/2012,31/12/2012,NaN,SI,NaN
1,Administración del Patrimonio de la Beneficenc...,F109-07,ADJUDICACION DIRECTA,HUERTA MÉDICA MEXICANA S.A. DE C.V.,21/06/2007,25254.0,APBP ADMINISTRACION DEL PATRIMONIO DE LA BENEF...,PRÓTESIS DE RODILLA,21/06/2007,25/06/2007,NaN,NaN,NaN
2,Administración del Patrimonio de la Beneficenc...,F108-07,ADJUDICACION DIRECTA,SMITH & NEPHEW S.A. DE C.V.,21/06/2007,22035.0,APBP ADMINISTRACION DEL PATRIMONIO DE LA BENEF...,PRÓTESIS DE RODILLA,21/06/2007,03/07/2007,NaN,NaN,NaN
3,Administración del Patrimonio de la Beneficenc...,F107-07,ADJUDICACION DIRECTA,STRYKER MÉXICO S.A. DE C.V.,21/06/2007,26450.0,APBP ADMINISTRACION DEL PATRIMONIO DE LA BENEF...,PRÓTESIS DE CADERA,21/06/2007,03/07/2007,NaN,NaN,NaN
4,Administración del Patrimonio de la Beneficenc...,F106-07,ADJUDICACION DIRECTA,BIOMET MÉXICO S.A. DE C.V.,21/06/2007,32969.93,APBP ADMINISTRACION DEL PATRIMONIO DE LA BENEF...,PRÓTESIS DE CADERA,21/06/2007,26/06/2007,NaN,NaN,NaN


In [8]:
proceds[proceds.CODIGO_CONTRATO.isin(imss['Número de Contrato'])]

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO
6,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,AZLA PUBLICIDAD S DE RL MI DE CV,HABILITADO,MICRO,74160.00,NaN,MXN,AA-901024986-N17-2012,None,ADJUDICACION DIRECTA FEDERAL,140948,ELABORACION DE PLAYERAS Y CACHUCHAS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-07-02 13:31:00,2012-07-04 09:00:00,NaT,NaT,2012-07-04,2012-08-31,207446,PLAYERAS Y GORRAS PARA ESCUELA SIEPRE ABIERTA,Z15122015 07. Adjudicación Directa Nacional Ar...,None,PoC,None,No,MX,NaN,None,NaN
210,APF,COMISION FEDERAL DE ELECTRICIDAD,CFE,CFE-ADMINISTRACION ZONA PASO DEL NORTE #018TOQ896,018TOQ896,ARNULFO SINUE GIRON GUERRERO,NO MIPYME,nan,COMPANIA REGIOMONTANA DE SERVICIOS SA DE CV,HABILITADO,PEQUENA,379000.00,NaN,MXN,IA-018TOQ896-N9-2012,Mixta,INVITACION A CUANDO MENOS 3 PERSONAS,100784,MANTENIMIENTO A TRANFORMADOR,None,SERVICIOS,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,2012-02-29 16:44:00,2012-03-08 13:00:00,2012-03-09,NaT,2012-03-13,2012-12-31,149981,MANTENIMIENTO A TRANSFORMADOR,Z15122015 04. Invitación a cuando menos tres N...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN
213,APF,COMISION FEDERAL DE ELECTRICIDAD,CFE,CFE-ADMINISTRACION ZONA PASO DEL NORTE #018TOQ896,018TOQ896,ARNULFO SINUE GIRON GUERRERO,MICRO,nan,EDITH CALDERON BRAVO,HABILITADO,MICRO,30000.00,NaN,MXN,AA-018TOQ896-N47-2012,Mixta,ADJUDICACION DIRECTA FEDERAL,140532,600323147 TRASLADO DE ACEITE REGENERADO,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-05-29 00:00:00,NaT,NaT,2012-05-29,2012-12-31,190676,600323147 TRASLADO DE ACEITE REGENERADO,Z15122015 07. Adjudicación Directa Nacional Ar...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN
215,APF,COMISION FEDERAL DE ELECTRICIDAD,CFE,CFE-ADMINISTRACION ZONA PASO DEL NORTE #018TOQ896,018TOQ896,ARNULFO SINUE GIRON GUERRERO,MICRO,nan,"JAIME A. DELGADO Y ASOCIADOS, S.C.",HABILITADO,None,38500.00,NaN,MXN,AA-018TOQ896-N64-2012,Mixta,ADJUDICACION DIRECTA FEDERAL,140466,ELABORACION DE COPIAS CERTIFICADAS 600325169 Z...,None,SERVICIOS,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-07-02 00:00:00,NaT,NaT,2012-06-22,2012-12-31,209895,ELABORACION DE COPIAS CERTIFICADAS 600325169 Z...,Z15122015 07. Adjudicación Directa Nacional Ar...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN
226,APF,COMISION FEDERAL DE ELECTRICIDAD,CFE,CFE-ADMINISTRACION ZONA PASO DEL NORTE #018TOQ896,018TOQ896,ARNULFO SINUE GIRON GUERRERO,MICRO,nan,"EFISA ELECTRONIC INJECTION, S.A. DE C.V.",HABILITADO,None,26200.00,NaN,MXN,AA-018TOQ896-N65-2012,Mixta,ADJUDICACION DIRECTA FEDERAL,140446,REPARACION VEHICULO OFICIAL N0. 94587 ZONA JUAREZ,None,SERVICIOS,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2012-07-02 00:00:00,NaT,NaT,2012-06-25,2012-12-31,209920,REPARACION VEHICULO OFICIAL N0. 94587 ZONA JUAREZ,Z15122015 07. Adjudicación Directa Nacional Ar...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN
232,APF,COMISION FEDERAL DE ELECTRICIDAD,CFE,CFE-ADMINISTRACION ZONA PASO DEL NORTE #018TOQ896,018TOQ896,ARNULFO SINUE GIRON GUERRERO,MICRO,nan,ERWIN RIVERA LUJAN,HABILITADO,None,46800.00,NaN,MXN,AA-018TOQ896-N34-2012,Mixta,ADJUDICACION DIRECTA FEDERAL,140210,600316855 PROTESIS AUDITIVA,None,ADQUISICIONES,EXPI

In [10]:
imss[imss['Número de Contrato']=='140948']

,Institución,Número de Contrato,Procedimiento de contratación,Nombre de la persona a que se asignó el contrato,Fecha de celebración del Contrato,Monto del Contrato,Unidad Administrativa que celebró el contrato,Objeto del Contrato,Fecha Inicio del Contrato,Fecha de terminación del contrato,Documento: Sitio en Internet,Convenio Modificatorio,CONTRATACIONES
26453,HOSPITAL GENERAL DE MÉXICO DR. EDUARDO LICEAGA,140948,ADJUDICACION DIRECTA,COMERCIALIZADORA JAS PER S.A. DE C.V.,24/10/2014,1825.0,DIRECTOR GENERAL,PRODUCTOS ALIMENTICIOS ANIMALES,24/10/2014,03/11/2014,NaN,NaN,NaN
71627,INSTITUTO NACIONAL DE CARDIOLOGÍA IGNACIO CHÁVEZ,140948,ADJUDICACION DIRECTA,JHADYD S.A. DE C.V.,26/08/2014,203.5,SUBDIRECCIÓN DE RECURSOS MATERIALES,MEDICINAS Y PRODUCTOS FARMACEUTICOS,28/08/2014,05/12/2014,NaN,NaN,NaN
100576,INSTITUTO NACIONAL DE CARDIOLOGÍA IGNACIO CHÁVEZ,140948,ADJUDICACION DIRECTA,JHADYD S.A. DE C.V.,26/08/2014,203.5,SUBDIRECCIÓN DE RECURSOS MATERIALES,MEDICINAS Y PRODUCTOS FARMACEUTICOS,28/08/2014,05/12/2014,NaN,NaN,NaN


In [11]:
proceds[proceds.CODIGO_CONTRATO=='140948']

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO
6,GE,_GOBIERNO DEL ESTADO DE AGUASCALIENTES,AGS,AGS-INSTITUTO DE EDUCACION DE AGUASCALIENTES-I...,901024986,ANTONIO RUBEN CIENFUEGOS BENAVIDES,NO MIPYME,nan,AZLA PUBLICIDAD S DE RL MI DE CV,HABILITADO,MICRO,74160.0,NaN,MXN,AA-901024986-N17-2012,None,ADJUDICACION DIRECTA FEDERAL,140948,ELABORACION DE PLAYERAS Y CACHUCHAS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,None,0.0,0.0,2012-07-02 13:31:00,2012-07-04 09:00:00,NaT,NaT,2012-07-04,2012-08-31,207446,PLAYERAS Y GORRAS PARA ESCUELA SIEPRE ABIERTA,Z15122015 07. Adjudicación Directa Nacional Ar...,None,PoC,None,No,MX,NaN,None,NaN


In [17]:
proceds[(proceds.PROVEEDOR_CONTRATISTA.str.find('JHADYD')!=-1) & (proceds.IMPORTE_CONTRATO<500)]

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO
166335,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,"JHADYD, S.A. DE C.V.",HABILITADO,None,139.2,NaN,MXN,AA-012NAW001-N106-2013,Electrónica,ADJUDICACION DIRECTA FEDERAL,506212,MEDICINAS Y PRODUCTOS FARMACEUTICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,NaN,NaT,2013-01-07,NaT,2013-08-22,2013-08-21,2013-08-26,557793,"MEDICINAS Y PRODUCTOS FARMACEUTICOS, 2013 AD",07. Adjudicación Directa Nacional Art. 42 LAASSP,None,UC,None,No,None,NaN,None,NaN
167371,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,"JHADYD, S.A. DE C.V.",HABILITADO,None,155.0,NaN,MXN,AA-012NAW001-N60-2012,Electrónica,ADJUDICACION DIRECTA FEDERAL,429858,MEDICINAS Y PRODUCTOS FARMACEUTICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,NaN,NaT,2011-12-27,NaT,2013-04-16,2013-04-15,2013-04-22,203285,MEDICINAS Y PRODUCTOS FARMACEUTICOS,07. Adjudicación Directa Nacional Art. 42 LAASSP,None,UC,None,No,None,NaN,None,NaN
394901,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,"JHADYD, S.A. DE C.V.",HABILITADO,None,360.0,NaN,MXN,AA-012NAW001-N50-2014,Electrónica,ADJUDICACION DIRECTA FEDERAL,649940,MEDICINAS Y PRODUCTOS FARMACEUTICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,2014-01-01,NaT,2014-06-20,2014-06-19,2014-06-26,618885,"MEDICINAS Y PRODUCTOS FARMACEUTICOS, 2014 AD",Z15122015 07. Adjudicación Directa Nacional Ar...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN
721514,APF,HOSPITAL JUAREZ DE MEXICO,HJM,HJM-HOSPITAL JUAREZ DE MEXICO #012NAW001,012NAW001,MARTIN ANTONIO MANRIQUE,PEQUENA,nan,"JHADYD, S.A. DE C.V.",HABILITADO,None,199.5,NaN,MXN,AA-012NAW001-E15-2016,Electrónica,ADJUDICACION DIRECTA FEDERAL,1023950,MEDICINAS Y PRODUCTOS FARMACEUTICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,NaN,0.0,NaT,NaT,NaT,2016-01-15,2016-01-15,2016-01-22,1017668,"MEDICINAS Y PRODUCTOS FARMACEUTICOS, AD 2016",05. Adjudicación Directa LAASSP,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN


In [21]:
proceds[(proceds.DEPENDENCIA.str.find('INSTITUTO NACIONAL DE CARDIOLOGIA')!=-1) & (proceds.IMPORTE_CONTRATO<500)]

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO
578522,APF,INSTITUTO NACIONAL DE CARDIOLOGIA IGNACIO CHAVEZ,INCARD,INCARD-SUBDIRECCION DE RECURSOS MATERIALES #01...,012NCA001,ALEJANDRO ALATORRE HERNANDEZ,NO MIPYME,nan,"COMPANIA INTERNACIONAL MEDICA, S.A. DE C.V.",HABILITADO,None,445.92,NaN,MXN,SA-012NCA001-N266-2015,Presencial,ADJUDICACION DIRECTA FEDERAL,861285,MEDICINAS Y PRODUCTOS FARMACEUTICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,NaT,NaT,NaT,2015-08-24,2015-12-31,899705,MEDICINAS Y PRODUCTOS FARMACEUTICOS,Z15122015 12. Adjudicación Directa Nacional Si...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN
578641,APF,INSTITUTO NACIONAL DE CARDIOLOGIA IGNACIO CHAVEZ,INCARD,INCARD-SUBDIRECCION DE RECURSOS MATERIALES #01...,012NCA001,ALEJANDRO ALATORRE HERNANDEZ,NO MIPYME,nan,"FRANWEY COMERCIALIZADORA, S.A DE C.V.",HABILITADO,None,117.00,NaN,MXN,LA-012NCA001-I12-2015,Presencial,LICITACION PUBLICA,764228,LICITACION INTERNACIONAL CONSUMIBLES DE COMPUT...,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Internacional,0.0,0.0,2015-03-31 13:25:00,2015-04-23 11:00:00,2015-04-30,NaT,2015-04-30,2015-12-31,791269,LICITACION CONSUMIBLES DE COMPUTO 2015,Z15122015 03. Licitación Pública Internacional...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,0.0
578934,APF,INSTITUTO NACIONAL DE CARDIOLOGIA IGNACIO CHAVEZ,INCARD,INCARD-SUBDIRECCION DE RECURSOS MATERIALES #01...,012NCA001,ALEJANDRO ALATORRE HERNANDEZ,MICRO,nan,ULISES JIMENEZ MORALES,HABILITADO,None,452.40,NaN,MXN,SA-012NCA001-N284-2015,Presencial,ADJUDICACION DIRECTA FEDERAL,861974,OTROS MATERIALES Y ARTICULOS DE CONSTRUCCION Y...,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,NaT,NaT,2015-08-17,2015-08-16,2015-08-31,900133,OTROS MATERIALES Y ARTICULOS DE CONSTRUCCION Y...,Z15122015 12. Adjudicación Directa Nacional Si...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN
578941,APF,INSTITUTO NACIONAL DE CARDIOLOGIA IGNACIO CHAVEZ,INCARD,INCARD-SUBDIRECCION DE RECURSOS MATERIALES #01...,012NCA001,ALEJANDRO ALATORRE HERNANDEZ,NO MIPYME,nan,COMPANIA INTERNACIONAL MEDICA S.A. DE C.V.,HABILITADO,None,148.64,NaN,MXN,SA-012NCA001-N275-2015,Presencial,ADJUDICACION DIRECTA FEDERAL,861535,MEDICINAS Y PRODUCTOS FARMACEUTICOS,None,ADQUISICIONES,EXPIRADO,0.0,0.0,Nacional,0.0,0.0,NaT,NaT,NaT,NaT,2015-08-02,2015-08-31,899873,MEDICINAS Y PRODUCTOS FARMACEUTICOS,Z15122015 12. Adjudicación Directa Nacional Si...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN
